In [1]:
#import libraries
# import libraries
import pandas as pd
import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


nltk.download("vader_lexicon")
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\UMUT\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\UMUT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\UMUT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\UMUT\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\UMUT\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
#veri setini yukle  duygu_analizi_amazon_veri_seti.csv
df = pd.read_csv("duygu_analizi_amazon_veri_seti.csv")

In [5]:
# text cleaning ve preprocessing
lemmatizer = WordNetLemmatizer()
def clean_preprocess_data(text):
    
    # 1. küçük harfe çevir ve tokenize et
    tokens = word_tokenize(text.lower())  
    # Örn: "This is a GOOD book." -> ['this', 'is', 'a', 'good', 'book', '.']
    
    # 2. stopwordleri çıkar (önemsiz kelimeler: is, the, and vs.)
    filtered_tokens = [token for token in tokens if token not in stopwords.words("english")]
    # Örn: ['good', 'book', '.']

    # 3. lemmatize et (kelimeleri kök forma indir)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    # Örn: ['good', 'book', '.'] (bazı kelimeler değişebilir: 'books' -> 'book')

    # 4. tekrar metne dönüştür
    processed_text = " ".join(lemmatized_tokens)
    # ['good', 'book'] -> "good book"
    
    return processed_text
    
# dataFrame'deki "reviewText" sütununa temizlik işlemi uygula
df["reviewText2"] = df["reviewText"].apply(clean_preprocess_data)


In [6]:
# sentiment analysis(nltk)
analyzer = SentimentIntensityAnalyzer()

# 1. Ardından, "get_sentiments" adında bir fonksiyon tanımlıyoruz.
# Bu fonksiyon, bir metin alıyor ve analiz edip pozitif mi değil mi diye karar veriyor.
def get_sentiments(text):
    # a. "polarity_scores" metodu, verilen metindeki olumlu, olumsuz, nötr ve genel duygu puanlarını döndürür.
    score = analyzer.polarity_scores(text)

    # b. Burada basit bir mantık kullanılmış: Eğer pozitif skor sıfırdan büyükse bu metin olumlu kabul ediliyor.
    #    Böylece sonuç ikili hale getirilmiş oluyor: 1 = pozitif, 0 = değil.
    sentiment = 1 if score["pos"] > 0 else 0

    return sentiment

# 2. Son olarak, elimizdeki metin verisine bu duygu analizini uyguluyoruz.
# "reviewText2" sütunundaki ön işlenmiş metinlere analiz fonksiyonu uygulanarak yeni bir sütun oluşturuluyor.
df["sentiment"] = df["reviewText2"].apply(get_sentiments)

# Bu işlem sonucunda, artık elimizde her bir yoruma ait bir duygu sınıflandırması (pozitif/negatif) olacak.
# Bu sınıflandırma, örneğin bir ürünün müşteri memnuniyetini analiz etme veya yorum filtreleme gibi alanlarda kullanılabilir.

In [7]:
# evaluation - test
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(df["Positive"], df["sentiment"])
print(f"Confusion matrix: {cm}")

Confusion matrix: [[ 1131  3636]
 [  576 14657]]


In [8]:
cr = classification_report(df["Positive"], df["sentiment"])

print(f"Classification report: \n{cr}")

Classification report: 
              precision    recall  f1-score   support

           0       0.66      0.24      0.35      4767
           1       0.80      0.96      0.87     15233

    accuracy                           0.79     20000
   macro avg       0.73      0.60      0.61     20000
weighted avg       0.77      0.79      0.75     20000

